In [ ]:
pip install autocorrect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622363 sha256=39778c9cd6721d1da9e4a2e75732806be8794495ee9f0d0c416a9de67d43d8b3
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect


In [ ]:
pip install gensim

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2023-10-20 14:05:27--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-10-20 14:05:27--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-10-20 14:05:27--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
!ls

glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip
glove.6B.200d.txt  glove.6B.50d.txt   sample_data


#Assignment 4: Word Embeddings for Shakespearean English
## COMP-3704 (241552) Neural Networks and Deep Learning

###Brandon, Santiago

## Objective
To train word embeddings on famous works of Shakespeare and evaluate their understanding.

##Data
The entire text of plays: 1) The Tragedy of Hamlet, Prince of Denmark, 2) The Tragedy of Macbeth, and 3) The Tragedy of Julius Caesar. These are available from the Gutenberg corpus of the NLTK library. Characters and synopses can be found on Wikipedia.

##Problem statement
Natural language processing is an important part of the most advanced artificial intelligence software we have today. By studying volumes of text, word embeddings are able to elicit meaning from the words within training data. Your goal is to train a word embedding on three famous works of Shakespeare to determine how well your embedding can understand the meaning of character names and other Shakespearean English words found in these plays.

## 1. Data

In [ ]:
# Importing Libraries
import nltk
import re

# download nltk packs, collab requires these be downloaded every run
nltk.download('stopwords')
nltk.download('gutenberg')
nltk.download('punkt')

# Importing specific functions
from nltk.stem.porter import *
from nltk.corpus import gutenberg, stopwords
from nltk import tokenize
from nltk.util import flatten
from nltk.lm import vocabulary


from gensim.scripts import glove2word2vec
from pprint import pprint
from gensim.models import Word2Vec, keyedvectors
from autocorrect import Speller
from tabulate import tabulate, SEPARATING_LINE

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


###Creating stop words and importing plays

In [ ]:
# Importing three plays
plays = [(gutenberg.raw('shakespeare-hamlet.txt')),
         (gutenberg.raw('shakespeare-macbeth.txt')),
         (gutenberg.raw('shakespeare-caesar.txt'))]

# Downloading stop word list from nltk
stop_words = stopwords.words('english')

# Added stop words from old english
stp_words = ['thy', 'thou', 'thee', 'thine', 'doth',
             'shalt', 'wert', 'wilt', 'oft', 'art',
             'like', 'would', 'know', 'make', 'hath',
             'ith', 'hast', 'though', 'oh', 'th', 'thats',
             'k', 'ii', 'whos', 'ther', 'tis', 'may', 'let',
             'therefore', "much", 'whose', 'made', 'shall',
             'come', 'vs','h', 'must', 'ought', 'qu', 'em','thi',
             'seems', 'theres', 'g', 'mee', 'al']

# Combining stop words
stp_words.extend(stop_words)

# Importing stemmer
stemmer = PorterStemmer()

# Setting autocorrect to english and making it do it fast
check = Speller(lang = 'en', fast = True)

### Cleansing the data

In [ ]:
# Tokenizing paragraphs to sentances, and setting sentances to lowercase
temp = flatten([tokenize.sent_tokenize(i.lower()) for i in plays])
# Tokenizes sentances to words
# Check is autocorrect, auto corrects current word
# stemmer.stem stems current word
# r'[^\w\s]', '' - removes punctuation
temp = [tokenize.word_tokenize(check(
                  stemmer.stem(re.sub(r'[^\w\s]', '',
                  i)))) for i in temp]

# Removes StopWords
temp = [[i for i in b if i not in stp_words] for b in temp]

In [ ]:
# Print first 5 sentances
i =0
while i < 5:
  print(temp[i])
  i+=1

['tragedies', 'hamlet', 'william', 'shakespeare', '1599', 'acts', 'prime']
['scoena', 'prima']
['enter', 'barnardo', 'francisco', 'two', 'sentinel']
['barnardo']
[]


By printing out the first 5 sentance, we were able to find a couple more words that could have been considered stop words in shakespear english. These were added to the list of stop words to remove

## 2. Modeling

### Function to display top 20 words

In [ ]:
# Function prints top 20 words
def top_20(model):

  # Saves word rank, word and frequency
  tab = [[count+1, word, model.wv.get_vecattr(word, 'count')] \
         # Enumerates list of words from index_to_key
         for count, word in enumerate(model.wv.index_to_key)\
         # Ensures only first 20 are used
         if count < 20]
  # Header names
  col_names = ['Rank', 'Word', 'Frequency']

  # prints tabulated results
  print(tabulate(tab, headers=col_names))

### Creating models

In [ ]:
# make the CBOW Word2Vex model
cbow_model = Word2Vec(
            sentences=temp,
            vector_size= 100,
            epochs= 20,
            window= 9,
            min_count= 5,
            sg= 0,
            workers = 4)

# save the model to a file
cbow_model.save('cbow_word2vec.model')

# loading the model
cbow_model = Word2Vec.load('cbow_word2vec.model')

# Displaying 20 most frequent words
top_20(cbow_model)

  Rank  Word      Frequency
------  ------  -----------
     1  ham             337
     2  lord            300
     3  enter           230
     4  good            218
     5  mac             205
     6  cesar           193
     7  well            173
     8  one             166
     9  von             155
    10  bru             153
    11  brutus          147
    12  self            140
    13  king            139
    14  go              138
    15  yet             136
    16  man             134
    17  speak           130
    18  say             128
    19  see             122
    20  give            118


From this list of top 20 words we have added a few to words to remove since they were words that did not add much context or meaning to sentances. Such as "vs".

In [ ]:
# make the skipgram Word2Vex model
skipgram_model = Word2Vec(
            sentences=temp,
            vector_size= 70,
            epochs= 20,
            window= 3,
            min_count= 5,
            sg= 1,
            workers = 6
)

# save the model to a file
skipgram_model.save('skipgram_word2vec.model')

# loading the model
skipgram_model = Word2Vec.load('skipgram_word2vec.model')

In [ ]:
glove_model = keyedvectors.load_word2vec_format('glove.6B.50d.txt',
                                                binary = False,
                                                no_header = True)

### GloVe model training

The GloVe model have beeen trained on the non-zero entries of a aggregated global word-word co-occurence matrix. This matrix tabulates how frequently words co-occur with one another in a given corpus

To populate the matrix we are required to do a single pass through the entire corpus, which is computationally expensive. But it is a one time cost

## 3. Discussion

### 3A

In [ ]:
# list the terms to find similar words later
terms = ['hamlet', 'cauldron', 'nature', 'spirit', 'general', 'prythee']

# find the most similar terms for each model
def most_similar_terms(models, term):
  result = {}
  for word, model in models.items():
    if model != glove_model:
      similar_terms = [similar for similar, _ in model.wv.most_similar(term, topn = 5)]
      result[word] = similar_terms

    elif model == glove_model and term != 'prythee':
      similar_terms = [similar for similar, _ in model.most_similar(term, topn = 5)]
      result[word] = similar_terms
  return result

# models to compare
models = {
    'CBOW     ': cbow_model,
    'Skip-gram': skipgram_model,
    'GloVe    ': glove_model
}

# find and display similar terms
for term in terms:
  print(f"Similar terms to '{term}':")
  similar_terms = most_similar_terms(models, term)
  for word, terms in similar_terms.items():
    print(f"{word}: {', '.join(terms)}")
  print()

Similar terms to 'hamlet':
CBOW     : queen, king, laertes, polonius, poloniu
Skip-gram: queen, laertes, polonius, horatio, king
GloVe    : village, situated, town, obec, sleepy

Similar terms to 'cauldron':
CBOW     : double, trouble, fire, burned, scale
Skip-gram: burned, appear, wisely, demand, directly
GloVe    : caldron, flame, flickering, brominated, cauldrons

Similar terms to 'nature':
CBOW     : still, might, even, stop, ore
Skip-gram: breath, oth, purpose, mortal, business
GloVe    : context, particular, aspects, aspect, unique

Similar terms to 'spirit':
CBOW     : fell, begin, yes, lay, swords
Skip-gram: power, motion, strength, sting, country
GloVe    : passion, desire, faith, essence, belief

Similar terms to 'general':
CBOW     : petty, capitol, base, hearts, begin
Skip-gram: wish, dangerous, hes, spoke, change
GloVe    : chief, secretary, vice, deputy, appointed

Similar terms to 'prythee':
CBOW     : cheese, mothers, grown, memory, fellow
Skip-gram: strata, humbly, tha

#### 'Hamlet'
Both cbow and skipgram appear to have captured people close to hamlet, and the fast that he was up for the throne so he would have been king. But we do not believe it captured the meaning of the word overall

Glove gave what appears to be random words related to the land/area hamlet took place since it gives us words such as village and town

#### 'Cauldron'
Cbow model has appeared to capture the attributes of a cauldren. Since it can burn things, witches use it to cause trouble and it requires a fire to use, and the scale of what you produce can be changed. overall it did not really capute the meaning of the word

skipgram also appears to have caputred some attributes of a cauldron but not the meaning

GloVe model appears to have captured best what a cauldron is, the words it said are most similar are good descriptioins of a caudron

#### 'Nature'
We believe cbow has captured a decent meaning of the word. Nature is still, and assuming that it is referring to the might of it, it has much might, it is even to all that live in it with its cycles of life and death

Skipgram model did capture the meaning of nature if you consider nature evil, oth means wicked, and it can be wicked with or without a purpose, and mortals live within it

GloVe model did not capture any meaning of nature whatsoever

##### 'Spirit'
We belive that the meaning of spirit was not captured by Cbow. Fell could be seen as felled but that refers to a person dying not a spirit, Begin could be considered since in macbeth spirits usually start some sort of trouble but the other 3 similar words just dont make sense

Skipgram captured more of a meaning for the word spirit. Spirits in the story have some level of power and strength, there presence alone scares people who see them. Seeing a spirit in the story usually results in something beinf set into motion.

GloVe model captured more if a joyous/insightful meaning to the word spirit.

#### 'General'
In the case of the stories, cbow captured the meaning is general fairly well. The generals tend to be petty, begin trouble, and represent the capitol to an extent.

Skipgram also captured a good meaning for general, they are dangerous, they are all men in the story (hes_), they wish for a change of some sort

Glove model agve the word general more of an overarching meaning, just what you think of when you normally think of one

### 'prythee'
both cbow and skipgram did not capture the meaning of the word prythee
prythee is a an informal and polite way to make a request or ask someone to do something

GloVe model did not conatin the word prythee so it was skipped


### 3B

In [ ]:
# list of cosine words
cosine_words = [['brutus', 'murder'],
                ['lady', 'macbeth', 'queen', 'gertrude'],
                ['fortinbras', 'norway'],
                ['rome', 'norway'],
                ['ghost', 'spirit'],
                ['macbeth', 'hamlet']]

# list containing models
models_cos = [cbow_model, skipgram_model, glove_model]

In [ ]:
cbow = []
skpgram = []
glove = []

# Loops through models
for q in range(len(models_cos)):

  # Checks if the loop is on the final iterationm since the final model is
  # the Glove model, and Glove model does not use wv
  if q != 2:

    # Loops through the cosine word list
    for i in range(len(cosine_words)):

      # one of the cosine word lists is longer than the others and
      # requires a extra step for cosine computation
      if len(cosine_words[i]) == 2:

        # Saves first and second wv to variable
        res1 = models_cos[q].wv[cosine_words[i][0]]
        res2 = models_cos[q].wv[cosine_words[i][1]]

      else:

        # Saves all 4 wv to different vairable
        v1 = models_cos[q].wv[cosine_words[i][0]]
        v2 = models_cos[q].wv[cosine_words[i][1]]
        v3 = models_cos[q].wv[cosine_words[i][2]]
        v4 = models_cos[q].wv[cosine_words[i][3]]

        # math
        res1 = (v1+v2)/2
        res2 = (v3+v4)/2

      # Computes cosine similarities between two words
      cosine = models_cos[q].wv.cosine_similarities(res1, [res2])

      # Checks what model program is currently on
      if models_cos[q] == cbow_model:

        # Appends to cbow list for tabulation later on
        cbow.append(cosine)
      else:

        #  Appends to skpgram list for tabulation later on
        skpgram.append(cosine)
  # runs same program as one above but without .wv
  # since keyedvectors has no attribute .wv
  else:

    # Loops through
    for i in range(len(cosine_words)):
      # one of the cosine word lists is longer than the others and
      # requires a extra step for cosine computation
      if len(cosine_words[i]) == 2:

        # Saves first and second wv to variable
        res1 = models_cos[q][cosine_words[i][0]]
        res2 = models_cos[q][cosine_words[i][1]]

      else:

        # Saves all 4 wv to different vairable
        v1 = models_cos[q][cosine_words[i][0]]
        v2 = models_cos[q][cosine_words[i][1]]
        v3 = models_cos[q][cosine_words[i][2]]
        v4 = models_cos[q][cosine_words[i][3]]

        # Math
        res1 = (v1+v2)/2
        res2 = (v3+v4)/2
      # Computes cosine similarities between two words
      cosine = models_cos[q].cosine_similarities(res1, [res2])
      # Appends to glove list for tabulation later on
      glove.append(cosine)

In [ ]:
# x represents the firs word
x = [['brutus'], ['lady macbeth'], ['fortinbras'],
     ['rome'], ['ghost'], ['macbeth']]

# y represents the second word, or the word being compared against the first word
y = [['murder'], ['queen gertrude'], ['norway'],
     ['norway'], ['spirit'], ['hamlet']]

# Merges all lists previously created for tabulation
num = list(zip(x, y,cbow,skpgram, glove))

# Headers
headers = ['First word', 'Second word', 'cbow cosine',
           'skipgram cosine', 'glove cosine']

# Prints tabulte table
print(tabulate(num, headers=headers, tablefmt='grid'))

+------------------+--------------------+---------------+-------------------+----------------+
| First word       | Second word        |   cbow cosine |   skipgram cosine |   glove cosine |
+==================+====================+===============+===================+================+
| ['brutus']       | ['murder']         |      0.936144 |          0.597817 |      0.0437047 |
+------------------+--------------------+---------------+-------------------+----------------+
| ['lady macbeth'] | ['queen gertrude'] |      0.972087 |          0.769663 |      0.780443  |
+------------------+--------------------+---------------+-------------------+----------------+
| ['fortinbras']   | ['norway']         |      0.997386 |          0.96293  |     -0.0363297 |
+------------------+--------------------+---------------+-------------------+----------------+
| ['rome']         | ['norway']         |      0.987662 |          0.813758 |      0.267536  |
+------------------+--------------------+---------

#### Cbow
cbow has the highest cosine between two words out of the three models. All of the cosine values are over 0.9.

Considering the data is multiple plays that are similar in story seeing that characters who are also similar have high cosine is pretty impressive (macbeth and hamlet, lady macbeth and queen gertrude).

The four cities all also have a high cosine which, and they are all from different plays

#### skipgram
Skipgram did not perform as well here. It gave a low cosine between brutus and murder.

Hamlet and macbeth were also given a fairly low cosine, despite that they are both characters that suffer from a tragedy in their given stories maybe it is from how they are refered to in each story?

Lady macbeth and queen gertrude had a fairly high cosine, which makes sense since they are both engaged/married to royalty and play pivitol roles involving the tragidy of the hero.

The four cities were given the highet cosine of them all.

Ghost and spirit were also given a fairly high cosine, despite the fact that they both act differently in each play. But we guess that since they are both just messangers in the story they are considered to be the same

#### GloVe
Glove just performed poorly, we believe that this is due to the fact that it was not trained on any data from the stories but rather how frequently words co appear with one another

### 3C

In [ ]:
linear_terms = [['denmark', 'queen'],
                ['scotland', 'army', 'general'],
                ['father', 'man', 'woman'],
                ['mother', 'woman', 'man']]

models_cos = [cbow_model, skipgram_model, glove_model]

sim = []
tmp = []
track = 0
for term in linear_terms:
  for i in models_cos:
    sum = 0
    for q in term:

      if i != glove_model:
        sum+=i.wv[q]
      else:
        sum+=i[q]

    if i == cbow_model:

      tmp = i.wv.most_similar(sum, topn=5)
      tmp_2 = ['cbow model']
      tmp_2.insert(0, '')

    elif i == skipgram_model:

      tmp = i.wv.most_similar(sum, topn=5)
      tmp_2 = ['skipgram model']
      tmp_2.insert(0, term)

    else:

      tmp = i.most_similar(sum, topn =5)
      tmp_2 = ['glove model']
      tmp_2.insert(0, '')

    for h in tmp:

      (u, m) = h

      tmp_2.append(u)

    sim.append(tmp_2)
    if i == glove_model:
      sim.append(SEPARATING_LINE)



headers = ['Words to compare', 'Model used', 'Most similar',
           '2nd place', '3rd place',
           '4th place','5th place',]
print(tabulate(sim, headers=headers, tablefmt="psql"))


+---------------------------------+----------------+----------------+-------------+-------------+-------------+-------------+
| Words to compare                | Model used     | Most similar   | 2nd place   | 3rd place   | 4th place   | 5th place   |
|---------------------------------+----------------+----------------+-------------+-------------+-------------+-------------|
|                                 | cbow model     | laertes        | polonius    | ophelia     | horatio     | lords       |
| ['denmark', 'queen']            | skipgram model | laertes        | polonius    | queen       | rosincrance | ophelia     |
|                                 | glove model    | queen          | denmark     | kingdom     | britain     | sweden      |
|  |
|                                 | cbow model     | yong           | ears        | soldier     | proud       | course      |
| ['scotland', 'army', 'general'] | skipgram model | titan          | cicero      | greefes     | mor         | 

#### 'Denmark' + 'Queen'
The Cbow model model listed the name of the royal family of denmark from the play hamlet, and horatio who is hamlets best friend. it also includes the word lords which is a good way to describe he relationship between queen and denmark. Overall it did a good job describing the idea behind the word vectors

The sg model also listed the royal family, but this time instead of listing horatio, it included Rosencrantz who was close to the royal family and was tasked with spying on hamlet. It also included the word queen instead of lords. Overall it did a good job describing the idea behind the word vectors

The GloVe model described where denmark was located and that it was a kingdom with. we dont think it did a good job describing the idea behind the word vectors

#### 'Scotland' + 'Army' + 'General'
The cbow model gave the words yong (youthfull), soldier, proud, course, and ears. The soldiers were all youthfull, proud and well soldiers. We would say that sbow described the idea behind the word vectors well.

the sg model gave some interesting results. Titan was autocorrected by the checker, We have no idea what it was before, cicero was most likely caeser who was a roman general, greefes means sorrow or troubles which a general and army woudl experience. We have no clue what to make of thank or mor. Overall it did an ok job at describing the idea behind the word vectors.

GloVe model gave words directly related to an army and generak such as command, force, and military. It did an excellent job describing the idea behind the word vecor

#### 'Father'+'Man'+'Woman"
The Cbow model gave the word sorrow, which given the plays and who would be xperiencing this it makes sense. Throne was also included, most of the people in the plays were royalty of sorts so this word also makes sense. Wife, the man/father usually had a wife in these tales, but the las two common and mans dont make sense to inlude. Oerall it did a decent job at describing the idea behind the word vectors

skipgram model gave some weird results that i cant really make sense of. Perhaps if we had read the plays they would make sense. Over all we would say it did a good job describing the idea behind the word vectors when taking what it was trained on into consideration

Glove Model just gave a couple general terms related to all three. man, woman, mother, father, son. Given the data it was trained on we would say that this was a good description of the word vectors, But weam surprised to not see family in that list of words

#### 'Mother'+'Woman'+'man
The Cbow model gave some some words that imply that these are related to a sudden (sodiene), sorrow, break in trust. withall means along with everything. With more context to the stories we could judge this better, but overall we will say that it did a good job at capturing the idea behinf the word vectors.


The skipgram model provided multiple words that lead us to believe that the women/mothers/men were guilt ridden, had betrayed or been the betrayor, and withall again, so been responsible for everything. Once again, reading the stories would help with understanding wether or not this could be conisdered a good description of the idea behind the word vectors. We would say that this model performed better than the cbow one since it gave us more descriptive words

The Glove mode gave some general words related to those three words and did not really capture the idea behind them.

### Conclusion

#### Cbow
The Cbow model performed great. It had the highet cosine score of the three models with all its cosine scores being over 0.9. but when it came to finding the meaning to one word, we found that it lacked the understanding of the word. When we compared it with multiple words it performed much better but still overall struggled to capture the idea behind word vectors.

#### Skipgram
The Skipgram model performed interstingly. It had cosine scores worse than the Cbow model but when it cam to actually find the meeaning behind a single word, we found that it outperformed the cbow model. When it came to finding the meaning behind multiple words it began to struggle and give us random words such as titan, land, bears, and withall.

#### GloVe
The Glove model performed well, its cosine scores were very bad, and when it came to giving descriptions regarding one or more word vectors. it gave a mixed performance.


### Imporving the models

We believe that including a couple more shakespeare tragidies, such as romeo and juliet, and othello would improve model performance. This would give the models more examples of works of shakespeare to look at and allow them to get a better understanding of the literature.

For the glove model, the only idea that we could think of would be to create a transfer learning model using the glove model weights and vocabulary as a base, then training the new model on 2-4 of shakespeare tragedy plays